--epoch', type=int, default=0, help='starting epoch

--n_epochs', type=int, default=200, help='number of epochs of training

--batchSize', type=int, default=10, help='size of the batches

--nes', type=str, default='data/nesmbd_tx1', help='root directory of the dataset

--lakh', type=str, default='data/5k_poprock_tx1', help='root directory of the dataset

--lr', type=float, default=0.0002, help='initial learning rate

--decay_epoch', type=int, default=100, help='epoch to start linearly decaying the learning rate to 0

--cuda', action='store_true', help='use GPU computation

--n_cpu', type=int, default=8, help='number of cpu threads to use during batch generation


In [3]:
!python train.py

Namespace(batchSize=10, cuda=False, decay_epoch=100, epoch=0, lakh='data/5k_poprock_tx1', lr=0.0002, n_cpu=8, n_epochs=200, nes='data/nesmdb_tx1')
Producing dataset nesmdb...
building vocab from data/nesmdb_tx1/vocab.txt
final vocab size 631
Loading cached dataset...
0it [00:00, ?it/s]
  5%|█▉                                      | 219/4502 [00:08<02:46, 25.76it/s]
0it [00:08, ?it/s]
Traceback (most recent call last):
  File "train.py", line 85, in <module>
    for i, ((nes, bptt), (lakh, _)) in enumerate(tqdm(data_stream)):
  File "/Users/dima/.local/lib/python3.7/site-packages/tqdm/std.py", line 1171, in __iter__
    for obj in iterable:
  File "/Users/dima/Documents/Github/CGanTransformers/data_utils.py", line 187, in __iter__
    trim_padding=self.trim_padding))
  File "/Users/dima/Documents/Github/CGanTransformers/utils/vocabulary.py", line 144, in encode_file
    encoded.append(self.convert_to_tensor(symbols))
  File "/Users/dima/Documents/Github/CGanTransformers/utils/vocabulary

# 1Hot to Mid

In [130]:
import torch
from models import Generator
import numpy as np
from data.tx1_midi import tx1_to_midi

In [21]:
checkpoint = torch.load('output/fullModel_0.pth')

In [69]:
nin_dims = 80
netG_A2B = Generator(nin_dims)
netG_A2B.load_state_dict(checkpoint['genBA'])

<All keys matched successfully>

In [71]:
X = torch.tensor(np.random.randint(631,size=(40,1)).astype(np.float32))

In [80]:
n_samples = 80
test_one_hot = np.eye(631)[np.random.choice(631, 80)]
test_one_hot = torch.tensor(test_one_hot.reshape((40,2,631)).astype(np.float32))

In [ ]:
#seq length, batch, 1hot_vector

In [132]:
out = netG_A2B.forward(test_one_hot)

In [133]:
torch.save(out,"test_tensor.pt")

In [4]:
with open('data/tx1_vocab.txt','r') as f:
    tokens = f.readlines()
    print(type(tokens[0]))
    tokens = [tok.strip('\n') for tok in tokens]
    idx2tok = {i:tokens[i-1] for i in range(1,631)}
idx2tok[0] = 'WT'

<class 'str'>


In [123]:
def oneHot_TX1(one_hot):
    argm = np.argmax(out.detach().numpy(),axis=2)
    batch_size = argm.shape[1]
    outputs = []
    for b in range(batch_size):
        toks = [idx2tok[int(i)] for i in argm[:,b]]
        outputs.append("\n".join(toks))
    return outputs
        
    
    

In [129]:
oneHot_TX1(out)

['WT_2900\nWT_3700\nWT_52000\nTR_NOTEON_48\nWT_370\nNO_NOTEON_8\nWT_2900\nWT_4400\nWT_2900\nWT_2900\nWT_2900\nWT_2900\nWT_2900\nWT_2900\nP2_NOTEON_33\nWT_2900\nWT_2900\nNO_NOTEOFF\nTR_NOTEON_63\nWT_2900\nWT_2900\nWT_2900\nWT_2900\nWT_2900\nWT_1\nWT_2200\nWT_2900\nWT_2900\nWT_1400\nWT_2900\nWT_4400\nP1_NOTEON_90\nP2_NOTEOFF\nWT_2900\nTR_NOTEON_100\nWT_1400\nP2_NOTEOFF\nWT_1000\nWT_2900\nTR_NOTEOFF',
 'WT_2900\nWT_2900\nWT_740\nWT_590\nP2_NOTEON_103\nWT_2900\nWT_2900\nP2_NOTEON_39\nWT_96\nWT_2900\nWT_2900\nWT_2900\nTR_NOTEON_65\nWT_600\nWT_870\nWT_460\nWT_2900\nWT_2900\nWT_2900\nWT_2900\nWT_1\nTR_NOTEON_96\nWT_730\nNO_NOTEON_13\nWT_3700\nP1_NOTEON_55\nWT_2900\nWT_63000\nWT_2900\nWT_10\nTR_NOTEOFF\nWT_2900\nWT_2900\nNO_NOTEON_13\nWT_2900\nWT_2900\nWT_2900\nP2_NOTEON_85\nWT_2900\nWT_47']

In [106]:
argm = np.argmax(out.detach().numpy(),axis=2)

In [107]:
argm.shape

(40, 2)

In [110]:
argm[:,1]

array([209, 209, 164, 149, 519, 209, 209, 455,  96, 209, 209, 209, 570,
       150, 177, 136, 209, 209, 209, 209,   1, 601, 163, 627, 217, 394,
       209, 333, 209,  10, 525, 209, 209, 627, 209, 209, 209, 501, 209,
        47])

In [112]:
aye = ['a','b','c']

In [118]:
print("\n".join(aye))

a
b
c


In [131]:
tx1_to_midi(oneHot_TX1(out)[0])

b'MThd\x00\x00\x00\x06\x00\x01\x00\x05V"MTrk\x00\x00\x00\x1d\x00\xffQ\x03\x07\xa1 \x00\xffX\x04\x04\x02\x18\x08\x87\xeeg\xffX\x04\x01\x00\x18\x08\x01\xff/\x00MTrk\x00\x00\x00\r\x00\xff\x03\x02p1\x00\xc0P\x01\xff/\x00MTrk\x00\x00\x00\x18\x00\xff\x03\x02p2\x00\xc1Q\x85\x8dV\x91!\x0f\x82\xa1\t!\x00\x01\xff/\x00MTrk\x00\x00\x00\'\x00\xff\x03\x02tr\x00\xc2&\x83\xc9h\x920\x01\x81\xf1\x160\x00\x00?\x01\x82\x8a5?\x00\x00d\x01\xa94d\x00\x01\xff/\x00MTrk\x00\x00\x00\x18\x00\xff\x03\x02no\x00\xc9y\x83\xccZ\x99\x08\x0f\x81\xee$\x08\x00\x01\xff/\x00'

# Tx1 to Mid

In [13]:
from data.tx1_midi import tx1_to_midi
fpathNoPerms = 'data/5k_poprock_tx1_noPerms/test/0a4f2051b572ebe007647fcab0114dd8_000.txt'
fpathPerms = 'data/5k_poprock_tx1/test/6e973cb0ce78fb872d9bbb9916c69f40_000.txt'
with open(fpathNoPerms,'r') as f:
    tx1 = f.read()
mid = tx1_to_midi(tx1)
mid.write('data/test_tx1_mid_in/no1hotNoPerm.mid')

In [14]:
mid.write("lol.mid")